In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/Kaggle/fashion_product_images"
%cd "/content/drive/MyDrive/Kaggle/fashion_product_images"

/content/drive/MyDrive/Kaggle/fashion_product_images


In [ ]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small
License(s): copyright-authors
fashion-product-images-small.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip \*.zip && rm *.zip

Streaming output truncated to the last 5000 lines.
  inflating: myntradataset/images/5813.jpg  
  inflating: myntradataset/images/58131.jpg  
  inflating: myntradataset/images/58132.jpg  
  inflating: myntradataset/images/58133.jpg  
  inflating: myntradataset/images/58135.jpg  
  inflating: myntradataset/images/58136.jpg  
  inflating: myntradataset/images/58137.jpg  
  inflating: myntradataset/images/58138.jpg  
  inflating: myntradataset/images/58139.jpg  
  inflating: myntradataset/images/5814.jpg  
  inflating: myntradataset/images/58140.jpg  
  inflating: myntradataset/images/58141.jpg  
  inflating: myntradataset/images/58143.jpg  
  inflating: myntradataset/images/58144.jpg  
  inflating: myntradataset/images/58145.jpg  
  inflating: myntradataset/images/58146.jpg  
  inflating: myntradataset/images/58147.jpg  
  inflating: myntradataset/images/58148.jpg  
  inflating: myntradataset/images/58149.jpg  
  inflating: myntradataset/images/5815.jpg  
  inflating: myntradataset/image

In [ ]:
# Import necessary libraries
from flask import Flask, request, jsonify
from flask_cors import CORS
import base64
from sklearn.neighbors import NearestNeighbors

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Load the model and precomputed features
with open('embeddings.pkl', 'rb') as f:
    feature_list = pickle.load(f)

with open('filenames.pkl', 'rb') as f:
    filenames = pickle.load(f)

feature_list = np.array(feature_list)
neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
neighbors.fit(feature_list)

def extract_features(img_path, model):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    return result

@app.route('/', methods=['GET'])
def home():
    return 'success', 200

@app.route('/find_similar', methods=['POST'])
def find_similar():
    if 'file' not in request.files:
        return 'No file part', 400

    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400

    if file:
        # Ensure 'uploads' directory exists
        if not os.path.exists('uploads'):
            os.makedirs('uploads')

        filepath = os.path.join('uploads', file.filename)
        file.save(filepath)

        feature = extract_features(filepath, model)
        distances, indices = neighbors.kneighbors([feature])

        similar_files = [filenames[i] for i in indices[0][1:6]]
        print(similar_files)
        # Convert images to base64
        similar_images_base64 = []
        for filename in similar_files:
            with open(filename, "rb") as image_file:
                encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
                similar_images_base64.append(encoded_string)

        return jsonify(similar_images_base64)

# Run Flask app with ngrok
app.run(port=80)

In [ ]:
!pip install flask flask-cors pyngrok

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
import pickle
from tqdm import tqdm

# Model setup
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tune the model
for layer in base_model.layers[:-10]:  # Freeze all layers except the last 10 layers
    layer.trainable = False

# Build the model by adding custom layers on top of the ResNet50 base
model = Sequential([
    base_model,
    GlobalMaxPooling2D(),  # Pooling layer to flatten the output from ResNet50
    Dense(512, activation='relu'),  # Optional Dense layer for fine-tuning
    Dropout(0.5)  # Dropout to prevent overfitting
])

# No need to compile the model for feature extraction
# model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Image directory and batch size
image_directory = 'images'  # Replace with your image directory
batch_size = 32

# Custom image generator without augmentation for feature extraction
def custom_image_generator(directory, batch_size=32, target_size=(224, 224)):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    num_files = len(files)

    while True:
        for offset in range(0, num_files, batch_size):
            batch_files = files[offset:offset + batch_size]
            batch_images = []

            for file in batch_files:
                img = load_img(file, target_size=target_size)
                img_array = img_to_array(img)
                img_array = preprocess_input(img_array)
                batch_images.append(img_array)

            batch_images = np.array(batch_images)
            yield batch_images, batch_files

# Generator initialization
generator = custom_image_generator(image_directory, batch_size=batch_size)

# Calculate number of batches
num_images = len([name for name in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, name))])
num_batches = np.ceil(num_images / batch_size)

# Initialize lists to store features and filenames
feature_list = []
filenames = []

# Extract features in batches
for _ in tqdm(range(int(num_batches))):
    batch_images, batch_files = next(generator)
    batch_features = model.predict(batch_images)
    batch_features = [feature.flatten() for feature in batch_features]  # Flatten features
    feature_list.extend(batch_features)
    filenames.extend(batch_files)

# Save features and filenames
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(feature_list, f)

with open('filenames.pkl', 'wb') as f:
    pickle.dump(filenames, f)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d_1 (Gl  (None, 2048)              0         
 obalMaxPooling2D)                                               
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
Total params: 24636800 (93.98 MB)
Trainable params: 5514752 (21.04 MB)
Non-trainable params: 19122048 (72.94 MB)
_________________________________________________________________


  0%|          | 0/1389 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


  0%|          | 1/1389 [01:13<28:19:22, 73.46s/it]

1/1 [==============================] - 1s 541ms/step


...

 26%|██▋       | 367/1389 [1:00:04<2:40:05,  9.40s/it]

1/1 [==============================] - 1s 558ms/step


 26%|██▋       | 368/1389 [1:00:13<2:39:43,  9.39s/it]

1/1 [==============================] - 1s 566ms/step


 27%|██▋       | 369/1389 [1:00:23<2:39:37,  9.39s/it]

In [ ]:
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
import pickle
import os
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)

# Load precomputed features and filenames
with open('embeddings.pkl', 'rb') as f:
    feature_list = pickle.load(f)
with open('filenames.pkl', 'rb') as f:
    filenames = pickle.load(f)

# Convert list of features to numpy array
feature_array = np.array(feature_list)

# Model setup
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers[:-10]:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalMaxPooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5)
])

# Define feature extraction function
def extract_features(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

# Define similarity function
def get_top_5_recommendations(image_features):
    # Compute cosine similarity between the input image features and precomputed features
    similarities = cosine_similarity([image_features], feature_array)
    # Get indices of top 5 most similar images
    top_indices = similarities[0].argsort()[-5:][::-1]
    return [filenames[i] for i in top_indices]

# Define API endpoint for recommendation
@app.route('/recommend', methods=['POST'])
def recommend():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No file selected'}), 400

    # Save the uploaded file
    file_path = 'temp_image.jpg'
    file.save(file_path)

    # Extract features from the uploaded image
    uploaded_image_features = extract_features(file_path)

    # Get top 5 recommendations
    recommendations = get_top_5_recommendations(uploaded_image_features)

    # Remove the temporary file
    os.remove(file_path)

    return jsonify({'recommendations': recommendations})

if __name__ == '__main__':
    app.run(debug=True)
